### SELECTING DBFS

In [0]:
%sql
SELECT * FROM delta.`/databricks-datasets/nyctaxi-with-zipcodes/subsampled`

### Listing files in DBFS using dbutils

In [0]:
files = dbutils.fs.ls("/databricks-datasets/nyctaxi-with-zipcodes/subsampled")
display(files)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS students 
  (id INT, name STRING, value DOUBLE)

In [0]:
%sql
INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
INSERT INTO students
VALUES 
  (4, "Ted", 4.7),
  (5, "Tiffany", 5.5),
  (6, "Vini", 6.3)

In [0]:
%sql
SELECT * FROM students

In [0]:
%python
assert spark.table("students").count() == 5, "The table should have 6 records"

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
UPDATE students 
SET value = value + 1
WHERE name LIKE "T%"

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
DELETE FROM students 
WHERE value > 6

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW updates(id, name, value, type) AS VALUES
  (2, "Omar", 15.2, "update"),
  (3, "", null, "delete"),
  (7, "Blue", 7.7, "insert"),
  (11, "Diya", 8.8, "update");
  
SELECT * FROM updates;

In [0]:
%sql
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *

In [0]:
%sql
SELECT * FROM students

In [0]:
%sql
DESCRIBE EXTENDED students

In [0]:
%sql
DESCRIBE DETAIL students

In [0]:
%sql
DESCRIBE HISTORY students

In [0]:
%sql
select * from students version as of 4

In [0]:
%sql


VACUUM students RETAIN 20000 HOURS DRY RUN

In [0]:
dataset_path = f"/Volumes/workspace/default/input"
print(dataset_path)

files = dbutils.fs.ls(dataset_path)
display(files)

In [0]:
%sql
SELECT * FROM json.`/Volumes/workspace/default/input/manifest-test .json`

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW manifest_view
AS SELECT * FROM json.`/Volumes/workspace/default/input/`;

SELECT * FROM manifest_view;

In [0]:
%sql
CREATE TABLE employee
USING CSV
OPTIONS (
  path 's3://adithya-databricks-demo/employee.csv',
  header 'true',
  delimiter ',',
  inferSchema 'true'
)

In [0]:
%sql
select * from employee;

In [0]:
%sql
DROP TABLE IF EXISTS abc_cyc_ctrl_tbl_jdbc;

CREATE TABLE abc_cyc_ctrl_tbl_jdbc
USING JDBC
OPTIONS (
  url = "jdbc:postgresql://usiipdnadb.cywxd5tina7v.us-west-2.rds.amazonaws.com:5432/postgres",
  dbtable = "abc_cyc_ctrl_tbl"
)

In [0]:
df = spark.read.format("jdbc").options(
    url="jdbc:postgresql://usiipdnadb.cywxd5tina7v.us-west-2.rds.amazonaws.com:5432/postgres",
    dbtable="abc_cyc_ctrl_tbl",
    user="postgresiip",
    password="usiipdnadb",
    driver="org.postgresql.Driver"
).load()

#df.createOrReplaceTempView("abc_cyc_ctrl_tbl_view")
display(df)

In [0]:
%sql
CREATE OR REPLACE TABLE employee_clone
DEEP CLONE employee

In [0]:
%sql
CREATE OR REPLACE TABLE employee_shallow_clone
SHALLOW CLONE employee

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW foods(food) AS VALUES
("beef"),
("beans"),
("potatoes"),
("bread");

SELECT * FROM foods;

CREATE FUNCTION foods_i_like(food STRING)
RETURNS STRING
RETURN CASE 
  WHEN food = "beans" THEN "I love beans"
  WHEN food = "potatoes" THEN "My favorite vegetable is potatoes"
  WHEN food <> "beef" THEN concat("Do you have any good recipes for ", food ,"?")
  ELSE concat("I don't eat ", food)
END;

In [0]:
%sql
select foods_i_like(food) from foods;

In [0]:
%sql
list 's3://adithya-databricks-demo'

In [0]:
(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", "/Volumes/workspace/default/employee_schema")
    .option("header", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .load("/Volumes/workspace/default/employee")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/workspace/default/employee_checkpoint")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("default.employee_delta")
)

In [0]:
%sql
select * from employee_delta

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE employee_delta
AS SELECT *
FROM STREAM read_files(
  '/Volumes/workspace/default/employee',
  format => 'csv'
)

In [0]:
%sql
-- Create a Bronze table to store raw insurance data
CREATE TABLE bronze_insurance_data (
    policy_id INT,
    policyholder_name STRING,
    policy_type STRING,
    premium_amount DECIMAL(10, 2),
    claim_amount DECIMAL(10, 2),
    claim_date TIMESTAMP,
    created_at TIMESTAMP
)

In [0]:
%sql
-- Insert raw insurance data into the Bronze table
INSERT INTO bronze_insurance_data VALUES
(101, 'Alice Johnson', 'Health', 500.00, NULL, NULL, CURRENT_TIMESTAMP),
(102, 'Bob Smith', 'Auto', 300.00, 1500.00, '2025-01-15', CURRENT_TIMESTAMP),
(103, 'Charlie Brown', 'Home', 400.00, NULL, NULL, CURRENT_TIMESTAMP),
(104, 'David Wilson', 'Health', 600.00, 2000.00, '2025-02-10', CURRENT_TIMESTAMP);


In [0]:
%sql
-- Create a Silver table to store cleaned insurance data
CREATE TABLE silver_insurance_data AS
SELECT
    policy_id,
    policyholder_name,
    policy_type,
    premium_amount,
    claim_amount,
    claim_date
FROM bronze_insurance_data
WHERE premium_amount IS NOT NULL;  -- Example transformation: filter out records with null premium amounts


In [0]:
%sql
-- Create a Gold table to store aggregated insurance data
CREATE TABLE gold_insurance_summary AS
SELECT
    policy_type,
    COUNT(*) AS total_policies,
    SUM(premium_amount) AS total_premium,
    SUM(COALESCE(claim_amount, 0)) AS total_claims,
    AVG(COALESCE(claim_amount, 0)) AS average_claim
FROM silver_insurance_data
GROUP BY policy_type;  -- Example aggregation: summarize by policy type
